In [1]:
# 한글깨짐 방지 코드
import matplotlib.pyplot as plt
import platform

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 
    
%matplotlib inline

In [2]:
import pandas as pd
from konlpy.tag import Mecab   # 메캡을 활용
mecab = Mecab()

In [4]:
# SSG 모든 리뷰들 중 긍정 리뷰들만 모아서 다 가져옴.
# 긍부정의 기준은 긍정은 별점 4점이상, 부정은 3점 이하.
# 긍부정의 라벨링(1과 0) 진행. 긍정은 1 부정은 0.
df_pos = pd.read_csv('SSG_positive_reviews.csv')
df_pos.head()

,comment,date,star_rate,label
0,이거 진짜 최고죠 ㅎㅎ 야식 고싶을때 냉장고에서 하나 돌려먹음 싸고 좋아요,2020-12-14,5,1
1,거의 매번 주문하는 상품인데 은근슬쩍 가격이 올랐네요 가성비가 좀씩 떨어지는거같아요,2020-12-14,5,1
2,살짝 냄새가 났던 잘먹었씁니닿 ㅎㅎㅎㅎ,2020-12-13,5,1
3,음 그냥 나쁘진 않았어요,2020-12-13,4,1
4,저렴하지만 맛은 좋아요,2020-12-13,5,1


In [5]:
len(df_pos)

204773

# 긍정 리뷰들 Mecab 토큰화

In [6]:
# 코멘트 리스트화 for 토큰화(모든 코멘트들을 하나의 리스트에 담는다 )
df_list = []
for i in range(len(df_pos['comment'])):
    df_list.append(df_pos['comment'][i])

In [ ]:
df_list

In [7]:
# Mecab 토큰화
from konlpy.tag import Mecab
mecab = Mecab()

mecab_list = []

for i in range(len(df_list)):
    mecab_list.append(mecab.morphs(df_list[i]))
# print(mecab_list) 

In [11]:
# 이중리스트를 풀어줌
list_of_list = mecab_list
result = []
for item in list_of_list:
    result.extend(item)
mecab_list = result

In [1]:
mecab_list 

In [13]:
# 포스태깅 해주고 리스트 만들어줌
mecab_tag = []

for token in mecab_list:
    mecab_tag += mecab.pos(token)
# print(mecab_tag)

In [14]:
# 메캡 포스태깅을 데이터프레임화 해줌
data = pd.DataFrame(mecab_tag)

In [15]:
data.columns = ['text', 'tag']

In [16]:
data.head()

,text,tag
0,이거,NP
1,진짜,MAG
2,최고,NNG
3,죠,NNP
4,ㅎㅎ,IC


In [17]:
data.shape

(1998922, 2)

In [18]:
tags_count = data['tag'].value_counts()

In [19]:
tags_count

NNG                   691454
VA                    190245
IC                    142794
MAG                   137978
NNP                   111520
                       ...  
NNB+VCP+ETM+NNB+JX         1
EP+ETN+JKO                 1
NP+VCP+EP                  1
NP+JKG+NNG+JX              1
XSN+VCP                    1
Name: tag, Length: 186, dtype: int64

In [16]:
# 긍정 리뷰들의 유니크한 형태소를 확인
data['tag'].unique()

array(['NP', 'MAG', 'NNG', 'NNP', 'IC', 'JKO', 'JKB', 'NR', 'VV', 'VV+EC',
       'EC', 'VA', 'JX', 'NP+VCP+EC', 'MM', 'VV+EP', 'ETM', 'EP',
       'UNKNOWN', 'EF', 'VA+ETM', 'NNB', 'VCP+EC', 'XR', 'VV+EP+EC',
       'MAJ', 'XSN', 'VV+ETM', 'VA+EC', 'XPN', 'NNB+VCP+EC', 'SL',
       'NP+JX', 'VA+EP', 'XSA+ETM', 'VV+EF', 'EP+EC', 'VX', 'VCN+EC',
       'VV+ETN', 'XSA+EC', 'NA+ETM', 'NNBC', 'VA+EF', 'VV+ETN+JKB+JX',
       'EP+EP', 'NP+JKG+NNG+JKB', 'VA+ETN', 'NP+JKO', 'MAG+VCP', 'NP+VCP',
       'ETN', 'XSA', 'NNBC+VCP+EC', 'NNB+JKB', 'JC', 'NNG+ETN', 'XSA+EP',
       'JKB+JX', 'VA+EC+VX+EP', 'VCP+UNA', 'NNG+JX', 'VX+EC', 'VX+EP+EC',
       'XSA+EC+VX+EC', 'VCP+EC+VX+EC', 'VX+EF', 'MAG+VCP+EC', 'VCP+EF',
       'EP+ETM', 'VV+EC+VX', 'XSV+EC', 'NP+JKS', 'VV+EC+VX+EC',
       'VV+EC+VX+EP', 'JKQ', 'VV+EC+VX+ETN', 'NP+VCP+EF', 'NP+JKB',
       'VV+EP+EF', 'VA+EC+VX+ETM', 'JKS', 'VV+EC+VX+ETM', 'NNB+VCP+EF',
       'XSA+ETN', 'NP+VCP+EC+JX', 'EC+JX+JX', 'VCP', 'NP+JKG',
       'NP+JKG+NNBC+

In [20]:
# for문을 돌려 원하는 형태소들이 포함된 유니크한 형태소들을 뽑기 위해 리스트화 
unique_tags = ['NP', 'MAG', 'NNG', 'NNP', 'IC', 'JKO', 'JKB', 'NR', 'VV', 'VV+EC',
       'EC', 'VA', 'JX', 'NP+VCP+EC', 'MM', 'VV+EP', 'ETM', 'EP',
       'UNKNOWN', 'EF', 'VA+ETM', 'NNB', 'VCP+EC', 'XR', 'VV+EP+EC',
       'MAJ', 'XSN', 'VV+ETM', 'VA+EC', 'XPN', 'NNB+VCP+EC', 'SL',
       'NP+JX', 'VA+EP', 'XSA+ETM', 'VV+EF', 'EP+EC', 'VX', 'VCN+EC',
       'VV+ETN', 'XSA+EC', 'NA+ETM', 'NNBC', 'VA+EF', 'VV+ETN+JKB+JX',
       'EP+EP', 'NP+JKG+NNG+JKB', 'VA+ETN', 'NP+JKO', 'MAG+VCP', 'NP+VCP',
       'ETN', 'XSA', 'NNBC+VCP+EC', 'NNB+JKB', 'JC', 'NNG+ETN', 'XSA+EP',
       'JKB+JX', 'VA+EC+VX+EP', 'VCP+UNA', 'NNG+JX', 'VX+EC', 'VX+EP+EC',
       'XSA+EC+VX+EC', 'VCP+EC+VX+EC', 'VX+EF', 'MAG+VCP+EC', 'VCP+EF',
       'EP+ETM', 'VV+EC+VX', 'XSV+EC', 'NP+JKS', 'VV+EC+VX+EC',
       'VV+EC+VX+EP', 'JKQ', 'VV+EC+VX+ETN', 'NP+VCP+EF', 'NP+JKB',
       'VV+EP+EF', 'VA+EC+VX+ETM', 'JKS', 'VV+EC+VX+ETM', 'NNB+VCP+EF',
       'XSA+ETN', 'NP+VCP+EC+JX', 'EC+JX+JX', 'VCP', 'NP+JKG',
       'NP+JKG+NNBC+VCP+EF', 'VCN', 'VCN+ETM', 'NNG+JKO', 'VV+EC+VX+EF',
       'NNG+VCP+EC', 'NNG+VCP+EF', 'XSV', 'JX+VCP+EC', 'EP+EP+EC',
       'VCP+EP+EC', 'XSA+EC+VX+ETN', 'NNB+JKS', 'VV+JKO',
       'VV+ETM+NNB+VCP', 'VV+EC+JX', 'VA+ETM+NNB', 'IC+ETN', 'NNB+JX',
       'MAG+JX', 'MAG+XSV+EP+EC', 'VX+ETM', 'VA+ETM+NNB+JKS', 'VCP+ETN',
       'NA+EP', 'VA+EP+EF', 'NP+VCP+EP', 'VCN+EF', 'EC+VX+EP+EC',
       'NNBC+JX', 'EC+VCP+EC', 'VA+EC+VCP+EC', 'VV+EC+EC', 'VCP+ETM',
       'NNG+XSV+ETM', 'EP+EP+EF', 'XSA+EC+VX+ETM', 'VV+EP+ETM',
       'NP+JKB+JX', 'VA+EP+EC', 'VA+ETN+JKB+JX', 'EP+ETN+JKO', 'NNG+EP',
       'NNB+VCP', 'VA+ETM+NNB+XSA+ETM', 'VA+ETM+NNB+XSA+EC',
       'ETM+NNB+XSA+EF', 'EC+VX+ETN', 'VA+EC+VX+EC', 'NP+JKG+NNG+JKS',
       'MAG+XSV+EC', 'MAG+VCP+ETM', 'XSV+EP', 'MM+NNB+JX', 'VV+EP+EP',
       'VA+EC+JX', 'EC+VX+EC', 'VA+EP+ETM', 'EC+JX', 'NNBC+JKO', 'MM+EC',
       'VV+EC+VX+EP+EC', 'NNP+JX', 'JKQ+JX', 'VA+EC+VX+EF',
       'VCP+ETN+VCP+EC', 'NNB+JKO+JX', 'VV+EP+EC+VV+EP+EC', 'EC+JX+VX+EP',
       'EC+VX', 'VX+EP', 'VA+ETM+NNB+JKB', 'NP+JKG+NNG+JX', 'XSN+VCP',
       'VV+EP+ETM+NNB+VCP', 'XSV+ETM', 'VV+EC+VCP+EC', 'NR+JX',
       'VV+EC+EP', 'NR+JKS', 'XSN+VCP+ETM', 'VX+ETN', 'VV+ETM+NNG+JX',
       'NNB+VCP+ETM+NNB+JX', 'NNB+JKO', 'NP+JKG+NNB+JX', 'XSN+VCP+EC',
       'VA+EC+VX+ETN', 'VA+EP+EP', 'NR+VCP+EC', 'NP+JKG+NNB',
       'VCP+EC+VV+EP+ETM', 'ETM+NNB+XSA+EC', 'XSV+EC+JKO', 'VCP+EP',
       'JKB+VCP+EC', 'IC+JX', 'XR+XSA+EC+VX+EC']

In [21]:
# 우리가 쓸 품사들을 가져오기 위한 추출작업: 명사, 긍정지정사, 부정지정사, 감탄사, 형용사, 동사, 어근
NN_resultlist = []
for tag in unique_tags:
    if "NN" in tag:
        NN_resultlist.append(tag)
        
VCP_resultlist = []
for tag in unique_tags:
    if "VCP" in tag:
        VCP_resultlist.append(tag)
        
VCN_resultlist = []
for tag in unique_tags:
    if "VCN" in tag:
        VCN_resultlist.append(tag)
        
IC_resultlist = []
for tag in unique_tags:
    if "IC" in tag:
        IC_resultlist.append(tag)
        
VA_resultlist = []
for tag in unique_tags:
    if "VA" in tag:
        VA_resultlist.append(tag)

VV_resultlist = []
for tag in unique_tags:
    if "VV" in tag:
        VV_resultlist.append(tag)

XR_resultlist = []
for tag in unique_tags:
    if "XR" in tag:
        XR_resultlist.append(tag)

In [22]:
# 각 리스트들을 다 합함
sum_tags_list = NN_resultlist + VCP_resultlist + VCN_resultlist + IC_resultlist + VA_resultlist + VV_resultlist + XR_resultlist

In [23]:
# 품사들의 중복을 제거하려함
final_tags_list = list(set(sum_tags_list))

In [24]:
# 그래서 우리가 확인할 태그 최종 목록
final_tags_list

['MAG+VCP+EC',
 'VCP+EC+VX+EC',
 'VCP+EC',
 'VA+EP',
 'NNB+VCP+EC',
 'NP+VCP',
 'VV+EC+VX',
 'VV+EC+EP',
 'JKB+VCP+EC',
 'VV+EC+JX',
 'NNG+ETN',
 'VV+EC+VX+ETN',
 'VA+ETM+NNB+XSA+EC',
 'VCN+ETM',
 'VV+EP+EC',
 'NP+JKG+NNG+JKB',
 'VA+EC+VCP+EC',
 'NNG+VCP+EF',
 'NNBC+JX',
 'VA+ETM',
 'VA+EP+EP',
 'NP+VCP+EC+JX',
 'VV+ETM',
 'VCP+UNA',
 'VV+EC',
 'VA+ETN',
 'NNBC+JKO',
 'XSN+VCP+EC',
 'XR+XSA+EC+VX+EC',
 'EC+VCP+EC',
 'VV+EP+EF',
 'MM+NNB+JX',
 'VV+JKO',
 'NNG+XSV+ETM',
 'VV+EC+VX+EP+EC',
 'XSN+VCP',
 'NNG+JX',
 'MAG+VCP',
 'VCN+EF',
 'ETM+NNB+XSA+EC',
 'VV+EC+VX+EP',
 'VCP+EF',
 'VA',
 'VA+EC',
 'NNG+EP',
 'NP+VCP+EC',
 'VA+EC+VX+EP',
 'VA+EC+VX+ETN',
 'VCP',
 'VA+EC+JX',
 'VV+EP+ETM',
 'VV+EP+EP',
 'JX+VCP+EC',
 'VV+ETM+NNG+JX',
 'NNB+JKB',
 'NNG+VCP+EC',
 'NNB+VCP+ETM+NNB+JX',
 'NP+JKG+NNB+JX',
 'ETM+NNB+XSA+EF',
 'VCP+EP',
 'NNB',
 'VA+ETM+NNB+JKS',
 'NR+VCP+EC',
 'VV+EC+VX+EC',
 'VV+EP+ETM+NNB+VCP',
 'XSN+VCP+ETM',
 'VA+ETN+JKB+JX',
 'VA+EF',
 'NNB+JKO',
 'XR',
 'VA+EP+EC',
 'NP+JKG

In [25]:
len(final_tags_list)

118

In [26]:
# 중복 제거한 품사들의 확인 리스트를 데이터프레임하기 위함
for_DataFrame = []
for i in mecab_tag:
    if i[1] in final_tags_list:
        for_DataFrame.append(i)

In [27]:
new_DataFrame = pd.DataFrame(for_DataFrame, columns=['text', 'tag'])

In [28]:
# n그램 하기 위해 text 들을 리스트화 해줌
for_ngram_list = data['text'].tolist()

In [29]:
# 최빈 단어들을 세기 위해 모듈 불러옴
from collections import Counter 

In [30]:
cols = ['words', 'count']

In [31]:
# 5-gram 형식으로 만듦
the_5gram_list = []
text = for_ngram_list
for i in range(len(text)-4):
    # print(text[i], text[i+1], text[i+2], text[i+3], text[i+4], sep='')
    the_5gram_list.append(text[i]+text[i+1]+text[i+2]+text[i+3]+text[i+4])

In [32]:
cnt5 = Counter(the_5gram_list) 
pos_reviews_most_common_words_5gram = cnt5.most_common()

In [33]:
# 긍정리뷰들의 5그램 최빈 표현들
pos_reviews_most_common_words_5gram

[('좋아요좋아요좋', 7372),
 ('아요좋아요좋아요', 7222),
 ('어요맛있어요', 3800),
 ('맛있어요맛있어', 2462),
 ('아이들이좋아', 2437),
 ('저렴하게잘', 2012),
 ('좋아요맛있어요', 1909),
 ('는제품이에요', 1836),
 ('좋아하는과자', 1675),
 ('구매하는제품입니다', 1592),
 ('먹고있어요', 1544),
 ('맛있게잘먹었', 1485),
 ('하고맛있어요', 1432),
 ('구매하는상품입니다', 1386),
 ('잘받았어요', 1368),
 ('잘먹었어요', 1308),
 ('맛있어요좋아요', 1297),
 ('하는제품이에', 1248),
 ('들이좋아해요', 1194),
 ('자주구매하는상품', 1126),
 ('이좋아하는', 1078),
 ('요맛있어요맛있', 1026),
 ('항상구매하는제품', 1024),
 ('하고있어요', 1012),
 ('구매하는제품이', 978),
 ('들이좋아하', 962),
 ('자주구매하는제품', 953),
 ('요좋아요좋아요', 931),
 ('게잘먹었습니다', 913),
 ('하게먹기좋', 902),
 ('게먹을수있', 820),
 ('들이좋아해서', 816),
 ('홈런볼맛있어요', 780),
 ('습니다잘받았습니다', 748),
 ('먹을수있어', 745),
 ('어요잘받았', 744),
 ('어요저렴하', 742),
 ('잘받았습니다감사', 740),
 ('저렴하고맛있', 729),
 ('받았습니다감사합니다', 727),
 ('게먹기좋아요', 724),
 ('게잘먹었어', 724),
 ('는제품이예요', 723),
 ('어요좋아요좋', 721),
 ('잘받았습니다잘', 720),
 ('늘구매하는제품', 716),
 ('간편하게먹기', 714),
 ('는상품이에요', 707),
 ('잘먹고있어', 704),
 ('저렴하게구매', 689),
 ('좋아요저렴하', 680),
 ('렴하게잘샀어요', 669),
 ('애들이좋아', 664),
 (

In [34]:
# 최빈 표현들을 데이터프레임화 해줌
df_pos_reviews_most_common_words_5gram = pd.DataFrame(pos_reviews_most_common_words_5gram,columns = cols )

In [35]:
# 데이터프레임을 csv 파일로 저장해줌  
df_pos_reviews_most_common_words_5gram.to_csv("SSG_pos_reviews_5gram", index=False, encoding="utf-8-sig")

In [ ]:
df5 = pd.read_csv("SSG_neg_reviews_5gram")

In [ ]:
df5.head(30)

In [ ]:
df4 = pd.read_csv("SSG_neg_reviews_4gram")

In [ ]:
df4.head(30)

In [ ]:
ㅇㄴㅏㅓㅣㄴㅁㅇㅏㅣㄷㅇㅑㅐㅓ ㅑ


In [ ]:
import matplotlib.pyplot as plt
import platform

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 
    
%matplotlib inline

